### For more explain insight at 'Data_Challenges_Insight-Sasiprapa_N.pdf'

In [1]:
import pandas as pd
import numpy as np 
from IPython.display import display
import re

In [2]:
## read csv file
df = pd.read_csv('supermarket_data.csv'\
                 , sep=','\
                 , encoding='utf-8'\
                 , dtype={'SHOP_DATE': 'str', 'SHOP_HOUR': 'int', 'BASKET_ID': 'str' }\
#                  , parse_dates=['SHOP_DATE']
                 )
pd.set_option('display.max_columns', None)

df

,SHOP_DATE,SHOP_HOUR,BASKET_ID,CUST_CODE,STORE_CODE,PROD_CODE,QUANTITY,SPEND
0,20071006,21,994107800268406,CUST0000153531,STORE00001,PRD0901391,1,0.37
1,20070201,15,994104300305853,CUST0000219191,STORE00002,PRD0901915,1,5.08
2,20071103,13,994108200514137,CUST0000526979,STORE00003,PRD0903379,1,2.36
3,20070206,18,994104400743650,CUST0000913709,STORE00004,PRD0903305,1,0.20
4,20071015,19,994108000780959,CUST0000961285,STORE00001,PRD0903387,1,1.65
...,...,...,...,...,...,...,...,...
671909,20080130,21,994109500479712,CUST0000501799,STORE00003,PRD0902839,1,2.08
671910,20071218,19,994108900475341,CUST0000399653,STORE00003,PRD0900205,3,5.91
671911,20071220,20,994108900278513,CUST0000128681,STORE00001,PRD0900722,4,48.40
671912,20071122,12,994108500434632,CUST0000412413,STORE00001,PRD0901855,1,2.96


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 671914 entries, 0 to 671913
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   SHOP_DATE   671914 non-null  object 
 1   SHOP_HOUR   671914 non-null  int64  
 2   BASKET_ID   671914 non-null  object 
 3   CUST_CODE   671914 non-null  object 
 4   STORE_CODE  671914 non-null  object 
 5   PROD_CODE   671914 non-null  object 
 6   QUANTITY    671914 non-null  int64  
 7   SPEND       671914 non-null  float64
dtypes: float64(1), int64(2), object(5)
memory usage: 41.0+ MB


In [4]:
df.describe()

,SHOP_HOUR,QUANTITY,SPEND
count,671914.000000,671914.000000,671914.000000
mean,14.745869,1.514344,1.852796
std,3.551738,1.668037,2.589564
min,8.000000,1.000000,0.010000
25%,12.000000,1.000000,0.750000
50%,15.000000,1.000000,1.210000
75%,17.000000,1.000000,2.040000
max,21.000000,73.000000,189.630000


In [ ]:
print ('Number of date in dataset: ', len(df.SHOP_DATE.unique()))
print ('Number of month in dataset: ', len(df['SHOP_DATE'].str[:-2].unique()))
df['shop_month'] = df['SHOP_DATE'].str[:-2]
print ('\tbetween ', df['SHOP_DATE'].max(), '-', df['SHOP_DATE'].min())

Number of date in dataset:  553


In [ ]:
## Drop 200701
df = df[df['shop_month']!='200807'].reset_index()
df = df.drop(columns='index')
print (df['SHOP_DATE'].max(), '-', df['SHOP_DATE'].min())

In [ ]:
print ('Number of customers: ', len(df.CUST_CODE.unique()))
print ('Number of times the purchase was made:', len(df.BASKET_ID.unique()))

In [ ]:
print ('Number of stores: ', len(df.STORE_CODE.unique()))
print ('Number of goods: ', len(df.PROD_CODE.unique()))

In [ ]:
print ('Top 100 best sellers:\n', df.groupby('PROD_CODE')['QUANTITY'].sum().sort_values(ascending=False)[:100])

In [ ]:
df_prod1 = df.groupby('PROD_CODE')['QUANTITY'].sum().reset_index()
df_prod1

In [ ]:
df_prod2 = df.groupby('PROD_CODE')['SPEND'].sum().reset_index()
df_prod2

In [ ]:
df_prod3 = df.groupby('PROD_CODE')['BASKET_ID'].unique().reset_index()
df_prod3['cnt_purchase'] = df_prod3['BASKET_ID'].apply(lambda x: len(x))
df_prod3 = df_prod3.drop(columns='BASKET_ID')
df_prod3

In [ ]:
df_prod = df_prod1.merge(df_prod2, how='inner', on='PROD_CODE')
df_prod['price'] = df_prod['SPEND']/df_prod['QUANTITY']
df_prod

In [ ]:
df_prod = df_prod.merge(df_prod3, how='inner', on='PROD_CODE')
df_prod

In [ ]:
df_prod.boxplot(column=['price'])

In [ ]:
df_prod.loc[df_prod['price'].idxmax()]

In [ ]:
df_prod.loc[df_prod['QUANTITY'].idxmax()]

In [ ]:
df_prod.loc[df_prod['cnt_purchase'].idxmax()]

In [ ]:
## get Quartile 3 of QUANTITY
## Value greater than Q3 of QUANTITY, it happens often

q3_quan_prod = np.quantile(df.groupby('PROD_CODE')['QUANTITY'].sum(), .75)
q3_quan_prod

In [ ]:
df_prod_f1 = df.groupby(['PROD_CODE', 'shop_month'])['QUANTITY'].sum().reset_index()
df_prod_f1_q3 = df_prod_f1[df_prod_f1['QUANTITY']>=q3_quan_prod].reset_index()
df_prod_f1_q3 = df_prod_f1_q3.rename(columns={'QUANTITY':'volumn'})
df_prod_f1_q3 = df_prod_f1_q3.drop(columns='index')
df_prod_f1_q3

In [ ]:
len(df_prod_f1_q3['PROD_CODE'].unique())

In [ ]:
df_prod_f1_q3.loc[df_prod_f1_q3['volumn'].idxmax()]

In [ ]:
import numpy as np; np.random.seed(0)
import seaborn as sns
import matplotlib.pyplot as plt

loss_cir_m = df_prod_f1_q3.pivot( "shop_month",'PROD_CODE', "volumn")
plt.figure(figsize=(15,10))

sns.heatmap(loss_cir_m, cmap="Blues")



In [ ]:
df_prod_f1_q3_piv = df_prod_f1_q3.pivot( "shop_month",'PROD_CODE', "volumn")
df_prod_f1_q3_piv

In [ ]:
prod_in_daily=[]
prod_m=[]
for i in df_prod_f1_q3_piv.columns.tolist():
    if df_prod_f1_q3_piv[i].notnull().all():
        prod_in_daily.append(i)
    else:
        prod_m.append(i)
print('Product in Daily Life:\n', prod_in_daily)

In [ ]:
df_prod_f1_q3_piv2 = df_prod_f1_q3.pivot( 'PROD_CODE', "shop_month", "volumn")

df_prod_f1_q3_piv2

In [ ]:
prod_hot_in_month={}
prod=[]
for i in df_prod_f1_q3_piv2.columns.tolist():
    for j in prod_m:
        tmp_v=[]
        tmp = df_prod_f1_q3[(df_prod_f1_q3['PROD_CODE']==j) & (df_prod_f1_q3['shop_month']==i)]['volumn'].isnull()
        if tmp.empty==False:
            if prod_hot_in_month.get(i) is not None:
                tmp_v = prod_hot_in_month[i]
                tmp_v.append(j)
            else:
                tmp_v.append(j)

            prod_hot_in_month[i] = tmp_v

print('Product hot:\n', prod_hot_in_month)

In [ ]:
df_prod_hot_in_month = pd.DataFrame(prod_hot_in_month.items(), columns=['date', 'product_cd'])
df_prod_hot_in_month

In [ ]:
df_prod_hot_in_month.to_csv('df_prod_hot_in_month.csv')

In [ ]:
df_cust1 = df.groupby('CUST_CODE')['BASKET_ID'].count().reset_index()
df_cust1 = df_cust1.rename(columns={'BASKET_ID':'freq_buy'})
df_cust1

In [ ]:
df_cust2 = df.groupby('CUST_CODE')['SPEND'].sum().reset_index()
df_cust2 = df_cust2.rename(columns={'SPEND':'total_spending'})
df_cust2

In [ ]:
df_cust = df_cust2.merge(df_cust1, how='inner', on='CUST_CODE')
df_cust

In [ ]:
df_cust['avg_spending'] = df_cust['total_spending']/df_cust['freq_buy']
df_cust

In [ ]:
df_cust3 = df.groupby(['CUST_CODE', 'STORE_CODE'])['BASKET_ID'].unique().reset_index()
df_cust3['BASKET_ID'] = df_cust3['BASKET_ID'].apply(lambda x: len(x))
df_cust3 = df_cust3.pivot(index='CUST_CODE', columns='STORE_CODE', values='BASKET_ID').fillna(0)
df_cust3

In [ ]:
df_cust = df_cust.merge(df_cust3, how='inner', on='CUST_CODE')
df_cust

In [ ]:
df_cust['visit_store'] = df_cust['STORE00001']+df_cust['STORE00002']+df_cust['STORE00003']+df_cust['STORE00004']
df_cust

In [ ]:
df_cust.loc[df_cust['total_spending'].idxmax()]

In [ ]:
df_cust.loc[df_cust['avg_spending'].idxmax()]

In [ ]:
df_cust.loc[df_cust['visit_store'].idxmax()]

In [ ]:
df_store1 = df.groupby(['STORE_CODE'])['BASKET_ID'].unique().reset_index()
df_store1['total_basket'] = df_store1['BASKET_ID'].apply(lambda x: len(x))
df_store1 = df_store1.drop(columns='BASKET_ID')
df_store1

In [ ]:
df_store1_tmp = df.groupby('STORE_CODE')['SPEND'].sum().reset_index()
df_store = df_store1.merge(df_store1_tmp, how='inner', on='STORE_CODE')
df_store = df_store.rename(columns={'SPEND':'total_spending'})
df_store

In [ ]:
num_month = len(df['SHOP_DATE'].str[:-2].unique())
df_store['avg_cust_spend_per_month'] = df_store['total_spending']/num_month
df_store['avg_cust_spend_per_person'] = df_store['total_spending']/df_store['total_basket']
df_store

In [ ]:
df_store_m1 = df.groupby(['STORE_CODE', 'shop_month'])['CUST_CODE'].unique().reset_index()
df_store_m1['total_cust'] = df_store_m1['CUST_CODE'].apply(lambda x: len(x))
df_store_m1 = df_store_m1.drop(columns='CUST_CODE')
df_store_m1

In [ ]:
df_store_m2 = df.groupby(['STORE_CODE', 'shop_month'])['SPEND'].sum().reset_index()
df_store_m2 = df_store_m2.rename(columns={'SPEND':'total_spend'})
df_store_m2

In [ ]:
df_store_m = df_store_m1.merge(df_store_m2, how='inner', on=['STORE_CODE', 'shop_month'])
df_store_m

In [ ]:
df_store_m3 = df.groupby(['STORE_CODE', 'shop_month'])['BASKET_ID'].unique().reset_index()
df_store_m3['total_basket'] = df_store_m3['BASKET_ID'].apply(lambda x: len(x))
df_store_m3 = df_store_m3.drop(columns='BASKET_ID')
df_store_m3

In [ ]:
df_store_m = df_store_m.merge(df_store_m3, how='inner', on=['STORE_CODE', 'shop_month'])
df_store_m

In [ ]:
df_store_m[df_store_m['STORE_CODE']=='STORE00001']

In [ ]:
import datetime
df['day_of_week'] = df['SHOP_DATE'].apply(lambda x: datetime.datetime.strptime(x, '%Y%m%d').strftime('%A'))
df

In [ ]:
df_freq = df.groupby(['STORE_CODE', 'day_of_week', 'SHOP_HOUR'])['CUST_CODE'].unique().reset_index()
df_freq['cnt_cust'] = df_freq['CUST_CODE'].apply(lambda x: len(x))
df_freq = df_freq.drop(columns='CUST_CODE')
df_freq

In [ ]:
for i in df_freq['STORE_CODE'].unique():
    tmp = df_freq[df_freq['STORE_CODE']==i]

    loss_cir_m = tmp.pivot("day_of_week",'SHOP_HOUR', "cnt_cust")
    plt.figure(figsize=(15, 10))
    plt.title(i)
    sns.heatmap(loss_cir_m, cmap="Blues")


In [ ]:
tmp = df_freq.drop(columns='STORE_CODE')
tmp = tmp.groupby(['day_of_week', 'SHOP_HOUR'])['cnt_cust'].sum().reset_index()
tmp
loss_cir_m = tmp.pivot("day_of_week",'SHOP_HOUR', "cnt_cust")
plt.figure(figsize=(15, 10))
plt.title('All Store')
sns.heatmap(loss_cir_m, cmap="Blues")

In [ ]:
df_freq_m = df.groupby(['STORE_CODE', 'shop_month', 'day_of_week', 'SHOP_HOUR'])['CUST_CODE'].unique().reset_index()
df_freq_m['cnt_cust'] = df_freq_m['CUST_CODE'].apply(lambda x: len(x))
df_freq_m = df_freq_m.drop(columns=['CUST_CODE', 'SHOP_HOUR'])
df_freq_m


In [ ]:
for i in df_freq_m['STORE_CODE'].unique():
    tmp = df_freq_m[df_freq_m['STORE_CODE']==i]\
                                .groupby(['shop_month', 'day_of_week']).sum().reset_index()
    loss_cir_m = tmp.pivot("day_of_week","shop_month", "cnt_cust")
    plt.figure(figsize=(15, 10))
    plt.title(i)
    sns.heatmap(loss_cir_m, cmap="Blues")
        

